In [435]:
# https://en.wikipedia.org/wiki/Delta_rule
import numpy as np
from tqdm import tqdm
#from prettytable import PrettyTable

In [436]:
from sklearn import datasets
iris = datasets.load_iris()

In [437]:
# config
weight = [1, 1, 1, 1]
learning_rate = 0.01
iterations = 200 * len(iris.data)
split = int(len(iris.data) / 5)

In [438]:
# dataset
dataset_full = []
for i in range(len(iris.target)):
    # add target to iris data
    dataset_full.append([iris.target[i], iris.data[i][0], iris.data[i][1], iris.data[i][2], iris.data[i][3]])

# shuffle
np.random.shuffle(dataset_full)
#dataset_full[:10]

In [439]:
# training set
X = []
for i in range(len(dataset_full[:-split])): X.append(dataset_full[i][1:])
X_t = []
for i in range(len(dataset_full[:-split])): X_t.append(dataset_full[i][0])
    
assert(len(X) == len(X_t))

In [440]:
# test set
Y = []
for i in range(len(dataset_full[-split:])): Y.append(dataset_full[i][1:])
Y_t = []
for i in range(len(dataset_full[-split:])): Y_t.append(dataset_full[i][0])
    
assert(len(Y) == len(Y_t))

In [441]:
# transfer function
# https://en.wikipedia.org/wiki/Heaviside_step_function
#def transfer_function(w, x, max_class=1, half_maximum=True):
#    wx = np.dot(w, x)
#    if (wx > 0):
#        return 1 * max_class
#    elif (wx == 0):
#        return 0.5 * max_class
#    else:
#        return 0
    
#assert (transfer_function([0.1, -0.5, 0.4], [0.1, -0.5, 0.4]) == 1)
#assert (transfer_function([0.1, -0.5, 0.4], [0.1, 0.5, 0.4]) == 0)
#assert (transfer_function([0, 0, 0], [0, 0, 0]) == 0.5)

def transfer_function(w, x, max_class=2):
    wx = np.dot(w, x)
    if (wx > 1):
        return 1 * max_class
    elif (wx == 0.5):
        return 0.5 * max_class
    else:
        return 0

In [442]:
# sequential delta learning algorithm
model = []
for o in tqdm(range(int(iterations / len(X)))):
    for i in range(len(X)):
        # break at max iterations
        if ((i + 1 + (len(X) * o)) > iterations): break

        weight_before = weight
        x = X[i]
        y = transfer_function(weight, x)

        # calculate update part
        update = np.zeros(len(x))
        for j in range(len(x)):
            update[j] = learning_rate * (X_t[i] - y) * x[j]
        
        # add update part to weight
        weight = np.add(weight, update)
        # append result
        model.append((
            # iteration
            str(i + 1 + (len(X) * o)),
            # weight before update
            np.round(weight_before, 4),
            # sample
            np.round(x, 4),
            # prediction
            int(np.round(y, 0)),
            # target
            np.round(X_t[i], 0),
            # new weight
            np.round(weight, 4)))

100%|██████████| 250/250 [00:02<00:00, 85.48it/s] 


In [443]:
print('iterations: ', model[-1][0])
print('weight before: ', list(model[-1][1]))
print('sample: ', list(model[-1][2]))
print('prediction (H(wx)): ', model[-1][3])
print('target: ', model[-1][4])
print('weight: ', list(model[-1][5]))
#print('training done? ', np.array_equal(model[-1][1], model[-1][5]))

iterations:  30000
weight before:  [-0.08, -0.32, 0.329, 0.966]
sample:  [5.0, 2.0, 3.5, 1.0]
prediction (H(wx)):  2
target:  1
weight:  [-0.13, -0.34, 0.294, 0.956]


In [444]:
weight_final = list(model[-1][1])
print(weight_final)
result = []
for i in range(len(Y)):
    x = Y[i]
    c = transfer_function(weight_final, x)
    # append to result
    result.append(c == Y_t[i])
    
print('Accuracy: ' + str(result.count(True) / len(result)))

[-0.08, -0.32, 0.329, 0.966]
Accuracy: 0.7333333333333333
